In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from mixed_naive_bayes import MixedNB
from sklearn.linear_model import LogisticRegression

In [6]:
df = pd.read_csv("../J1/HeartDiseaseUCI.csv", index_col=0)

In [7]:
num_variables = ["age", "trestbps", "chol", "thalach", "ca", "oldpeak"]
cat_variables = ["cp", "restecg", "thal"]
ord_variables = ["slope"]
bin_variables = ["sex", "fbs", "exang"]

In [8]:
df["target"] = np.where(df["num"] >= 1, 1, 0)

In [9]:
X = df.drop(["target", "num"], axis=1)
y = df["target"]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size = 0.2,
    stratify = df["target"],
    random_state = 314
)

In [11]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(242, 13)
(61, 13)
(242,)
(61,)


### 4. Logistic Regression

In [12]:
num_pipeline = Pipeline(steps = [
    ("imputer", KNNImputer()),
    ("scaler", MinMaxScaler())
])
cat_pipeline = Pipeline(steps = [
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(drop = "first"))
])
ord_pipeline = Pipeline(steps = [
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OrdinalEncoder())
])
bin_pipeline = Pipeline(steps = [
    ("imputer", SimpleImputer(strategy="most_frequent")),
])

In [13]:
preprocessor = ColumnTransformer(transformers=
     [
        ("num", num_pipeline, num_variables),
        ("cat", cat_pipeline, cat_variables),
        ("ord", ord_pipeline, ord_variables),
        ("bin", bin_pipeline, bin_variables),
     ],
 )

In [15]:
pipeline = Pipeline(steps = [
    ("preprocesor", preprocessor),
    ("logistic", LogisticRegression())
])

In [28]:
end_C = 1
step_C = 0.02
start_C = step_C

hyperparameters = [
    {
        "logistic__penalty" : ["l1", "l2", "elasticnet"],
        "logistic__C"       : np.arange(start_C, end_C, step_C),
        "logistic__max_iter": [10000],
        "logistic__solver"  : ["lbfgs", "liblinear", "newton-cg", "newton-cholesky", "sag", "saga"]
    },
    {
        "logistic__penalty" : [None],
        "logistic__max_iter": [10000],
        "logistic__solver"  : ["lbfgs", "liblinear", "newton-cg", "newton-cholesky", "sag", "saga"]
    }
]

In [29]:
gscv_logistic = GridSearchCV(
    estimator = pipeline,
    param_grid=hyperparameters,
    cv = 5,
    scoring = "f1",
    verbose = 4,
)

In [30]:
gscv_logistic.fit(X_train, y_train)

Fitting 5 folds for each of 888 candidates, totalling 4440 fits
[CV 1/5] END logistic__C=0.02, logistic__max_iter=10000, logistic__penalty=l1, logistic__solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END logistic__C=0.02, logistic__max_iter=10000, logistic__penalty=l1, logistic__solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END logistic__C=0.02, logistic__max_iter=10000, logistic__penalty=l1, logistic__solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END logistic__C=0.02, logistic__max_iter=10000, logistic__penalty=l1, logistic__solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END logistic__C=0.02, logistic__max_iter=10000, logistic__penalty=l1, logistic__solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END logistic__C=0.02, logistic__max_iter=10000, logistic__penalty=l1, logistic__solver=liblinear;, score=0.000 total time=   0.0s
[CV 2/5] END logistic__C=0.02, logistic__max_iter=10000, logistic__penalty=l1, logistic__solver=liblinear;, score=0.000 total time= 

[CV 4/5] END logistic__C=0.02, logistic__max_iter=10000, logistic__penalty=elasticnet, logistic__solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END logistic__C=0.02, logistic__max_iter=10000, logistic__penalty=elasticnet, logistic__solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END logistic__C=0.02, logistic__max_iter=10000, logistic__penalty=elasticnet, logistic__solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END logistic__C=0.02, logistic__max_iter=10000, logistic__penalty=elasticnet, logistic__solver=liblinear;, score=nan total time=   0.0s
[CV 3/5] END logistic__C=0.02, logistic__max_iter=10000, logistic__penalty=elasticnet, logistic__solver=liblinear;, score=nan total time=   0.0s
[CV 4/5] END logistic__C=0.02, logistic__max_iter=10000, logistic__penalty=elasticnet, logistic__solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END logistic__C=0.02, logistic__max_iter=10000, logistic__penalty=elasticnet, logistic__solver=liblinear;, score=nan total time= 

[CV 3/5] END logistic__C=0.04, logistic__max_iter=10000, logistic__penalty=l2, logistic__solver=lbfgs;, score=0.766 total time=   0.0s
[CV 4/5] END logistic__C=0.04, logistic__max_iter=10000, logistic__penalty=l2, logistic__solver=lbfgs;, score=0.791 total time=   0.0s
[CV 5/5] END logistic__C=0.04, logistic__max_iter=10000, logistic__penalty=l2, logistic__solver=lbfgs;, score=0.791 total time=   0.0s
[CV 1/5] END logistic__C=0.04, logistic__max_iter=10000, logistic__penalty=l2, logistic__solver=liblinear;, score=0.933 total time=   0.0s
[CV 2/5] END logistic__C=0.04, logistic__max_iter=10000, logistic__penalty=l2, logistic__solver=liblinear;, score=0.769 total time=   0.1s
[CV 3/5] END logistic__C=0.04, logistic__max_iter=10000, logistic__penalty=l2, logistic__solver=liblinear;, score=0.792 total time=   0.1s
[CV 4/5] END logistic__C=0.04, logistic__max_iter=10000, logistic__penalty=l2, logistic__solver=liblinear;, score=0.723 total time=   0.1s
[CV 5/5] END logistic__C=0.04, logistic

[CV 4/5] END logistic__C=0.06, logistic__max_iter=10000, logistic__penalty=l1, logistic__solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END logistic__C=0.06, logistic__max_iter=10000, logistic__penalty=l1, logistic__solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END logistic__C=0.06, logistic__max_iter=10000, logistic__penalty=l1, logistic__solver=liblinear;, score=0.732 total time=   0.0s
[CV 2/5] END logistic__C=0.06, logistic__max_iter=10000, logistic__penalty=l1, logistic__solver=liblinear;, score=0.750 total time=   0.1s
[CV 3/5] END logistic__C=0.06, logistic__max_iter=10000, logistic__penalty=l1, logistic__solver=liblinear;, score=0.830 total time=   0.1s
[CV 4/5] END logistic__C=0.06, logistic__max_iter=10000, logistic__penalty=l1, logistic__solver=liblinear;, score=0.731 total time=   0.1s
[CV 5/5] END logistic__C=0.06, logistic__max_iter=10000, logistic__penalty=l1, logistic__solver=liblinear;, score=0.720 total time=   0.1s
[CV 1/5] END logistic__C=0.06, logistic

[CV 3/5] END logistic__C=0.06, logistic__max_iter=10000, logistic__penalty=elasticnet, logistic__solver=liblinear;, score=nan total time=   0.0s
[CV 4/5] END logistic__C=0.06, logistic__max_iter=10000, logistic__penalty=elasticnet, logistic__solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END logistic__C=0.06, logistic__max_iter=10000, logistic__penalty=elasticnet, logistic__solver=liblinear;, score=nan total time=   0.0s
[CV 1/5] END logistic__C=0.06, logistic__max_iter=10000, logistic__penalty=elasticnet, logistic__solver=newton-cg;, score=nan total time=   0.0s
[CV 2/5] END logistic__C=0.06, logistic__max_iter=10000, logistic__penalty=elasticnet, logistic__solver=newton-cg;, score=nan total time=   0.0s
[CV 3/5] END logistic__C=0.06, logistic__max_iter=10000, logistic__penalty=elasticnet, logistic__solver=newton-cg;, score=nan total time=   0.0s
[CV 4/5] END logistic__C=0.06, logistic__max_iter=10000, logistic__penalty=elasticnet, logistic__solver=newton-cg;, score=nan tota

...

[CV 4/5] END logistic__C=0.9600000000000001, logistic__max_iter=10000, logistic__penalty=elasticnet, logistic__solver=newton-cg;, score=nan total time=   0.1s
[CV 5/5] END logistic__C=0.9600000000000001, logistic__max_iter=10000, logistic__penalty=elasticnet, logistic__solver=newton-cg;, score=nan total time=   0.0s
[CV 1/5] END logistic__C=0.9600000000000001, logistic__max_iter=10000, logistic__penalty=elasticnet, logistic__solver=newton-cholesky;, score=nan total time=   0.0s
[CV 2/5] END logistic__C=0.9600000000000001, logistic__max_iter=10000, logistic__penalty=elasticnet, logistic__solver=newton-cholesky;, score=nan total time=   0.1s
[CV 3/5] END logistic__C=0.9600000000000001, logistic__max_iter=10000, logistic__penalty=elasticnet, logistic__solver=newton-cholesky;, score=nan total time=   0.0s
[CV 4/5] END logistic__C=0.9600000000000001, logistic__max_iter=10000, logistic__penalty=elasticnet, logistic__solver=newton-cholesky;, score=nan total time=   0.0s
[CV 5/5] END logistic_

[CV 3/5] END logistic__C=0.98, logistic__max_iter=10000, logistic__penalty=l2, logistic__solver=newton-cg;, score=0.792 total time=   0.1s
[CV 4/5] END logistic__C=0.98, logistic__max_iter=10000, logistic__penalty=l2, logistic__solver=newton-cg;, score=0.780 total time=   0.0s
[CV 5/5] END logistic__C=0.98, logistic__max_iter=10000, logistic__penalty=l2, logistic__solver=newton-cg;, score=0.818 total time=   0.1s
[CV 1/5] END logistic__C=0.98, logistic__max_iter=10000, logistic__penalty=l2, logistic__solver=newton-cholesky;, score=0.909 total time=   0.0s
[CV 2/5] END logistic__C=0.98, logistic__max_iter=10000, logistic__penalty=l2, logistic__solver=newton-cholesky;, score=0.769 total time=   0.1s
[CV 3/5] END logistic__C=0.98, logistic__max_iter=10000, logistic__penalty=l2, logistic__solver=newton-cholesky;, score=0.792 total time=   0.0s
[CV 4/5] END logistic__C=0.98, logistic__max_iter=10000, logistic__penalty=l2, logistic__solver=newton-cholesky;, score=0.780 total time=   0.1s
[CV

[CV 4/5] END logistic__max_iter=10000, logistic__penalty=None, logistic__solver=newton-cg;, score=0.762 total time=   0.1s
[CV 5/5] END logistic__max_iter=10000, logistic__penalty=None, logistic__solver=newton-cg;, score=0.791 total time=   0.1s
[CV 1/5] END logistic__max_iter=10000, logistic__penalty=None, logistic__solver=newton-cholesky;, score=0.889 total time=   0.0s
[CV 2/5] END logistic__max_iter=10000, logistic__penalty=None, logistic__solver=newton-cholesky;, score=0.769 total time=   0.0s
[CV 3/5] END logistic__max_iter=10000, logistic__penalty=None, logistic__solver=newton-cholesky;, score=0.864 total time=   0.1s
[CV 4/5] END logistic__max_iter=10000, logistic__penalty=None, logistic__solver=newton-cholesky;, score=0.762 total time=   0.0s
[CV 5/5] END logistic__max_iter=10000, logistic__penalty=None, logistic__solver=newton-cholesky;, score=0.791 total time=   0.0s
[CV 1/5] END logistic__max_iter=10000, logistic__penalty=None, logistic__solver=sag;, score=0.889 total time=

/home/kf/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
2455 fits failed out of a total of 4440.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
245 fits failed with the following error:
Traceback (most recent call last):
  File "/home/kf/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/kf/.local/lib/python3.10/site-packages/sklearn/base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/home/kf/.local/lib/python3.10/site-packages/sklearn/pipeline.py", line 420, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  Fil

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocesor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          KNNImputer()),
                                                                                         ('scaler',
                                                                                          MinMaxScaler())]),
                                                                         ['age',
                                                                          'trestbps',
                                                                          'chol',
                                                                          'thalach',
                                                                          'ca',
                                                                          'oldpeak']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('encoder',
                                                                                          OneHotEncoder(drop='first'))]),
                                                                         ['cp',
                                                                          'restecg...
       0.9 , 0.92, 0.94, 0.96, 0.98]),
                          'logistic__max_iter': [10000],
                          'logistic__penalty': ['l1', 'l2', 'elasticnet'],
                          'logistic__solver': ['lbfgs', 'liblinear',
                                               'newton-cg', 'newton-cholesky',
                                               'sag', 'saga']},
                         {'logistic__max_iter': [10000],
                          'logistic__penalty': [None],
                          'logistic__solver': ['lbfgs', 'liblinear',
                                               'newton-cg', 'newton-cholesky',
                                               'sag', 'saga']}],
             scoring='f1', verbose=4)

In [32]:
best_logistic = gscv_logistic.best_estimator_
best_hyperparameters = gscv_logistic.best_params_

In [33]:
best_hyperparameters

{'logistic__C': 0.48000000000000004,
 'logistic__max_iter': 10000,
 'logistic__penalty': 'l1',
 'logistic__solver': 'liblinear'}

In [34]:
y_pred = best_logistic.predict(X_test)

In [35]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.85      0.80      0.82        35
           1       0.75      0.81      0.78        26

    accuracy                           0.80        61
   macro avg       0.80      0.80      0.80        61
weighted avg       0.81      0.80      0.80        61

